<h1 align='center'>
Lab 2 - Python Basics II
</h1>

<h3 align='center'>
Daniel Persaud

September 26th, 2022
</h3>

Import libraries necesary for this work

In [1]:
import os
import csv

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re
from mendeleev.fetch import fetch_table

import matminer
from matminer.datasets import load_dataset
from matminer.featurizers.conversions import StrToComposition
from matminer.featurizers.composition import ElementFraction
from matminer.featurizers.composition import ElementProperty
from matminer.featurizers.composition import BandCenter
from matminer.featurizers.conversions import CompositionToOxidComposition
from matminer.featurizers.composition import OxidationStates
from matminer.featurizers.composition import ElectronegativityDiff

/opt/anaconda3/envs/initStable/lib/python3.9/site-packages/pymatgen/analysis/phase_diagram.py:26: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


### <span style="color: red"> Task 1:</span> Some errors are presented in the code below. Can you identify and corret them?

In [2]:
current_dir = os.getcwd()                   # get current directory
filename = current_dir+'/E1_T1.csv'         # save path to data file
with open(filename, 'r', newline='') as f:  # open file read new line
    csv_file = csv.reader(f)                # save line to variable
    data = list(csv_file)                   # add line to list
sentence=''                                 # make an empty variable for the sentance
for word in data[0]:                        # for each word in the first line (only line)
    sentence = sentence + ' ' + word        # add the word to the sentance
print(sentence)                             # print the sentance
print(type(sentence))                       # print the type of the sentance

 This task is too simple , I want more challenges
<class 'str'>


### <span style="color: red"> Task 2:</span> Who has the most runner-up finishes in the histroy of NHL? Create a table with the teams and their runner-up finishes. The table must contain the the following columns:

-Team

-Total number of runner-up finishes

-Years [list of the years they finished as a runner-up]

You have to use Jupyter for the practice and produce a notebook with good readability.

### <span style="color: blue"> Solution:</span>

In [3]:
dfStanelyCupWinners = pd.read_excel('Stanely_cup_winners_by_year.xlsx') # use pandas to read xlsx file
dfStanelyCupWinners

,Year,Team,Runner up
0,2020,Tampa Bay Lightning,Dallas Stars
1,2019,St. Louis Blues,Boston Bruins
2,2018,Washington Capitals,Vegas Golden Knights
3,2017,Pittsburgh Penguins,Nashville Predators
4,2016,Pittsburgh Penguins,San Jose Sharks
...,...,...,...
103,1915,Vancouver Millionaires,Ottawa Senators
104,1914,Toronto Blueshirts,Victoria Cougars
105,1913,Quebec Bulldogs,Sydney Miners
106,1912,Quebec Bulldogs,Moncton Victories


Little bit of data cleaning

In [4]:
dfStanelyCupWinners.loc[dfStanelyCupWinners['Runner up'] == 'Chicago Black Hawks', 'Runner up'] = 'Chicago Blackhawks'  # force the 'Black Hawks' entry to be consistent with other entries

In [5]:
dfStanelyCupRunners = pd.DataFrame(columns = ['team','no_of_2nd_finishes','list_of_years_finished_2nd'])    # make an empty dataframe with the required columns
for index, row in dfStanelyCupWinners.iterrows():                                                           # for each row in the origional dataframe
    if (row['Runner up'] in (dfStanelyCupRunners['team'].values)):                                          # if the team in the row of the original dataframe is already in the new dataframe
        # add 1 to the count of the number of runner up finishes
        dfStanelyCupRunners.loc[dfStanelyCupRunners.index[dfStanelyCupRunners['team'] == row['Runner up']],'no_of_2nd_finishes'] = dfStanelyCupRunners.loc[dfStanelyCupRunners.index[dfStanelyCupRunners['team'] == row['Runner up']],'no_of_2nd_finishes'] + 1
        # add the year to the list of years finished as runner up
        dfStanelyCupRunners.loc[dfStanelyCupRunners.index[dfStanelyCupRunners['team'] == row['Runner up']],'list_of_years_finished_2nd'] = dfStanelyCupRunners.loc[dfStanelyCupRunners.index[dfStanelyCupRunners['team'] == row['Runner up']],'list_of_years_finished_2nd'] + ', ' + str(row['Year'])
    else:                                                                                                   # if the team in the row of the original dataframe is new to the new datafram
        # make a temporary dataframe with the new team, 1 runner up finish and the year
        temp = pd.DataFrame({'team': row['Runner up'], 'no_of_2nd_finishes': 1, 'list_of_years_finished_2nd': str(row['Year'])},index = [0])
        dfStanelyCupRunners = pd.concat([dfStanelyCupRunners,temp], axis=0, ignore_index=True)              # add temp dataframe to the bottom of the new dataframe
dfStanelyCupRunners = dfStanelyCupRunners.sort_values('team')                                               # sort the new dataframe by team name
dfStanelyCupRunners = dfStanelyCupRunners.reset_index(drop=True)                                            # reset index
dfStanelyCupRunners

,team,no_of_2nd_finishes,list_of_years_finished_2nd
0,Boston Bruins,14,"2019, 2013, 1990, 1988, 1978, 1977, 1974, 1958..."
1,Buffalo Sabres,2,"1999, 1975"
2,Calgary Flames,2,"2004, 1986"
3,Calgary Tigers,1,1924
4,Carolina Hurricanes,1,2002
5,Chicago Blackhawks,7,"1992, 1973, 1971, 1965, 1962, 1944, 1931"
6,Dallas Stars,2,"2020, 2000"
7,Detroit Red Wings,13,"2009, 1995, 1966, 1964, 1963, 1961, 1956, 1949..."
8,Edmonton Eskimos,1,1923
9,Edmonton Oilers,2,"2006, 1983"


### <span style="color: red"> Task 3:</span> With the help of matminer, 1. Retrive the 'steel_strength' dataset into a dataframe and clean your dataset in needed; 2. Featurize the compositions of the materials; and 3. Perform correlation analysis and drop feature(s) with an efficient greater than abs(0.8) from your dataframe.

### <span style="color: blue"> Solution:</span>

<h3 align='center'>
Load and clean dataset
</h3>

In [6]:
dfSteelStrength = load_dataset('steel_strength')    #load dataset from matminer
dfSteelStrength

,formula,c,mn,si,cr,ni,mo,v,n,nb,co,w,al,ti,yield strength,tensile strength,elongation
0,Fe0.620C0.000953Mn0.000521Si0.00102Cr0.000110N...,0.02,0.05,0.05,0.01,19.70,2.95,0.01,0.00,0.01,15.00,0.00,0.15,1.55,2411.5,2473.5,7.0
1,Fe0.623C0.00854Mn0.000104Si0.000203Cr0.147Ni0....,0.18,0.01,0.01,13.44,0.01,3.01,0.46,0.04,0.01,19.46,2.35,0.04,0.00,1123.1,1929.2,8.0
2,Fe0.625Mn0.000102Si0.000200Cr0.0936Ni0.129Mo0....,0.00,0.01,0.01,8.67,13.45,0.82,0.01,0.00,0.01,13.90,0.00,0.39,0.57,1736.3,1871.8,NaN
3,Fe0.634C0.000478Mn0.000523Si0.00102Cr0.000111N...,0.01,0.05,0.05,0.01,17.70,3.95,0.01,0.00,0.01,15.00,0.00,0.13,1.47,2487.3,2514.9,9.0
4,Fe0.636C0.000474Mn0.000518Si0.00101Cr0.000109N...,0.01,0.05,0.05,0.01,19.40,1.45,0.01,0.00,0.01,14.90,0.00,0.13,1.55,2249.6,2315.0,8.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,Fe0.823C0.0176Mn0.00183Si0.000198Cr0.0779Ni0.0...,0.38,0.18,0.01,7.27,0.01,3.77,0.96,0.00,0.01,4.90,0.00,0.03,0.00,1722.5,1998.1,5.0
308,Fe0.823Mn0.000618Si0.00101Cr0.0561Ni0.0984Mo0....,0.00,0.06,0.05,5.15,10.20,3.20,0.01,0.00,0.01,0.01,0.00,0.03,0.09,1019.0,1019.0,19.0
309,Fe0.825C0.0174Mn0.00175Si0.000201Cr0.0565Ni0.0...,0.37,0.17,0.01,5.20,0.01,5.84,1.05,0.00,0.91,4.88,0.00,0.03,0.00,1860.3,2156.6,3.0
310,Fe0.858C0.0191Mn0.00194Si0.000199Cr0.0753Ni0.0...,0.41,0.19,0.01,6.99,0.01,5.84,0.92,0.00,0.01,0.02,0.00,0.03,0.00,1812.1,2087.7,2.0


Cleaning: the matminer documentation states that the dataset is already cleaned but i'll just do some quick checks

In [7]:
dfSteelStrength['formula'].duplicated().any() # check that there arent any duplicate formulas

False

In [8]:
dfSteelStrength[dfSteelStrength.isna().any(axis=1)] # show all the entries with nan entries in the dataframe

,formula,c,mn,si,cr,ni,mo,v,n,nb,co,w,al,ti,yield strength,tensile strength,elongation
2,Fe0.625Mn0.000102Si0.000200Cr0.0936Ni0.129Mo0....,0.00,0.01,0.01,8.67,13.45,0.82,0.01,0.0,0.01,13.90,0.00,0.39,0.57,1736.3,1871.8,NaN
58,Fe0.678C0.000464Mn0.000102Si0.000199Cr0.0873Ni...,0.01,0.01,0.01,8.14,11.95,0.02,0.01,0.0,0.01,10.48,0.00,0.39,1.11,1768.0,1828.6,NaN
61,Fe0.680Mn0.000102Si0.000199Cr0.127Ni0.0976Mo0....,0.00,0.01,0.01,11.85,10.25,1.47,0.01,0.0,0.01,7.48,0.00,0.43,0.56,1586.7,1671.3,NaN
79,Fe0.686C0.000466Mn0.000102Si0.000199Cr0.0951Ni...,0.01,0.01,0.01,8.84,10.81,1.19,0.41,0.0,0.01,9.24,0.00,0.43,0.57,1755.2,1820.7,NaN
83,Fe0.687C0.000467Mn0.000102Si0.000200Cr0.0835Ni...,0.01,0.01,0.01,7.74,11.91,0.98,0.01,0.0,0.01,9.95,0.00,0.27,0.71,1779.7,1856.2,NaN
85,Fe0.687C0.0162Mn0.000101Si0.000199Cr0.0965Ni0....,0.35,0.01,0.01,9.00,1.50,2.00,0.30,0.0,0.01,18.00,0.00,0.03,0.02,1731.7,2232.4,NaN
86,Fe0.687Mn0.000102Si0.000199Cr0.119Ni0.0952Mo0....,0.00,0.01,0.01,11.11,10.00,1.23,0.01,0.0,0.01,7.51,0.28,0.57,0.59,1729.4,1817.2,NaN
112,Fe0.702Mn0.000102Si0.000200Cr0.154Ni0.0793Mo0....,0.00,0.01,0.01,14.30,8.30,2.60,0.01,0.0,0.01,4.30,0.00,0.10,0.49,1350.4,1478.0,NaN
113,Fe0.702Mn0.000102Si0.000200Cr0.154Ni0.0802Mo0....,0.00,0.01,0.01,14.30,8.40,2.50,0.01,0.0,0.01,4.30,0.00,0.12,0.47,1274.7,1347.0,NaN


The only entries with nan entries are missing elongation values - if elongation was the target of my ML task - would just drop these entries.

Since this is just an excersize I will pretend that yield strength is the property of interest so I won't be dropping / manipulating any entries.

<h3 align='center'>
Featurize
</h3>

I like the magpie features and since the task was not specified - it is as good as any other features so I will use them

The element property featurizer requires a list of the property the user wants for each entry and the statistics of that property - below, I have listed all the possible options

In [19]:
lstMagpieFeatures = ["Number", "MendeleevNumber", "AtomicWeight", "MeltingT", "Column", "Row", "CovalentRadius", "Electronegativity", "NsValence", "NpValence", "NdValence", "NfValence", "NValence", "NsUnfilled", "NpUnfilled", "NdUnfilled", "NfUnfilled", "NUnfilled", "GSvolume_pa", "GSbandgap", "GSmagmom", "SpaceGroupNumber"]
lstMagpieStats = ["minimum", "maximum", "range", "mean", "avg_dev", "mode"]

In [20]:
dfSteelStrength_features = dfSteelStrength[['formula']] # make a new dataset for just the features
dfSteelStrength_features

,formula
0,Fe0.620C0.000953Mn0.000521Si0.00102Cr0.000110N...
1,Fe0.623C0.00854Mn0.000104Si0.000203Cr0.147Ni0....
2,Fe0.625Mn0.000102Si0.000200Cr0.0936Ni0.129Mo0....
3,Fe0.634C0.000478Mn0.000523Si0.00102Cr0.000111N...
4,Fe0.636C0.000474Mn0.000518Si0.00101Cr0.000109N...
...,...
307,Fe0.823C0.0176Mn0.00183Si0.000198Cr0.0779Ni0.0...
308,Fe0.823Mn0.000618Si0.00101Cr0.0561Ni0.0984Mo0....
309,Fe0.825C0.0174Mn0.00175Si0.000201Cr0.0565Ni0.0...
310,Fe0.858C0.0191Mn0.00194Si0.000199Cr0.0753Ni0.0...


Featurize

In [21]:
dfSteelStrength_features = StrToComposition().featurize_dataframe(dfSteelStrength_features, "formula")
dfSteelStrength_features = ElementProperty('magpie',features = lstMagpieFeatures,stats = lstMagpieStats).featurize_dataframe(dfSteelStrength_features, col_id="composition")

StrToComposition:   0%|          | 0/312 [00:00<?, ?it/s]

ElementProperty:   0%|          | 0/312 [00:00<?, ?it/s]

In [22]:
dfSteelStrength_features = dfSteelStrength_features.drop(['formula', 'composition'], axis=1)

In [23]:
dfSteelStrength_features[dfSteelStrength_features.isna().any(axis=1)] # show all the entries with nan entries in the dataframe

,MagpieData minimum Number,MagpieData maximum Number,MagpieData range Number,MagpieData mean Number,MagpieData avg_dev Number,MagpieData mode Number,MagpieData minimum MendeleevNumber,MagpieData maximum MendeleevNumber,MagpieData range MendeleevNumber,MagpieData mean MendeleevNumber,...,MagpieData range GSmagmom,MagpieData mean GSmagmom,MagpieData avg_dev GSmagmom,MagpieData mode GSmagmom,MagpieData minimum SpaceGroupNumber,MagpieData maximum SpaceGroupNumber,MagpieData range SpaceGroupNumber,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber


<h3 align='center'>
Correlation Analysis
</h3>

In [24]:
dfCorrelationMatrix = dfSteelStrength_features.corr().abs()     # use pandas to calculate the absolute values of the correlations between features

In [27]:
# adapted from https://h1ros.github.io/posts/drop-highly-correlated-features/
threshold = 0.8                                                                         # set variable for the maximum correlation between features

columns = np.full((dfCorrelationMatrix.shape[0],), True, dtype=bool)                    # make a square matrix of true values for each feature, implying that we will start with all the columns
# this double for loop just ensures that correlations are taken from the upper triangle
for i in range(dfCorrelationMatrix.shape[0]):                                           # for each feature
    for j in range(i+1, dfCorrelationMatrix.shape[0]):                                  # with respect to each of the rest of the features
        if dfCorrelationMatrix.iloc[i,j] >= threshold:                                  # if the correlation is greater or equal to the threshold                      
            columns[j] = False                                                          # remove the correlated column
selected_columns = dfSteelStrength_features.columns[columns]                            # the columns to be used in the non-correlated dataframe are the ones still in the 'approved feature list'
dfSteelStrength_features_NonCorrelated = dfSteelStrength_features[selected_columns]     # take only the 'approved' columns from the original features
dfSteelStrength_features_NonCorrelated

,MagpieData minimum Number,MagpieData maximum Number,MagpieData mean Number,MagpieData avg_dev Number,MagpieData mode Number,MagpieData minimum MendeleevNumber,MagpieData maximum MendeleevNumber,MagpieData range MendeleevNumber,MagpieData mean MendeleevNumber,MagpieData avg_dev MendeleevNumber,...,MagpieData minimum GSmagmom,MagpieData maximum GSmagmom,MagpieData range GSmagmom,MagpieData mean GSmagmom,MagpieData mode GSmagmom,MagpieData minimum SpaceGroupNumber,MagpieData maximum SpaceGroupNumber,MagpieData range SpaceGroupNumber,MagpieData mean SpaceGroupNumber,MagpieData mode SpaceGroupNumber
0,6.0,42.0,26.664769,1.152116,26.0,43.0,78.0,35.0,56.377863,2.437556,...,0.0,2.110663,2.110663,1.648909,2.110663,194.0,229.0,35.0,222.420512,229.0
1,6.0,74.0,26.300744,1.522817,26.0,46.0,82.0,36.0,54.768497,2.013802,...,0.0,2.110663,2.110663,1.606402,2.110663,194.0,229.0,35.0,222.057381,229.0
2,13.0,42.0,26.145400,0.858372,26.0,43.0,78.0,35.0,55.653308,2.290349,...,0.0,2.110663,2.110663,1.600877,2.110663,194.0,229.0,35.0,223.594029,229.0
3,6.0,42.0,26.743298,1.235479,26.0,43.0,78.0,35.0,56.227153,2.327724,...,0.0,2.110663,2.110663,1.668280,2.110663,194.0,229.0,35.0,222.541850,229.0
4,6.0,42.0,26.526613,0.958219,26.0,43.0,78.0,35.0,56.375439,2.360988,...,0.0,2.110663,2.110663,1.677256,2.110663,194.0,229.0,35.0,222.528371,229.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,6.0,42.0,25.846224,1.069714,26.0,46.0,78.0,32.0,54.865014,1.324448,...,0.0,2.110663,2.110663,1.808822,2.110663,194.0,229.0,35.0,226.738291,229.0
308,13.0,42.0,26.362527,0.915270,26.0,43.0,78.0,35.0,55.178083,1.214834,...,0.0,2.110663,2.110663,1.795838,2.110663,194.0,229.0,35.0,228.553978,229.0
309,6.0,42.0,26.172381,1.331160,26.0,46.0,78.0,32.0,54.874724,1.302542,...,0.0,2.110663,2.110663,1.813977,2.110663,194.0,229.0,35.0,226.728508,229.0
310,6.0,42.0,25.971550,1.144885,26.0,46.0,78.0,32.0,54.717548,1.370770,...,0.0,2.110663,2.110663,1.811830,2.110663,194.0,229.0,35.0,228.298093,229.0


### <span style="color: red"> Task 4:</span> Setup a private Github repository for this course. Name it MSE1065_\<YourName>_\<YourStudentID>. You will be using this repo for homework submission and course project version control. A good place to get started is [here](https://docs.github.com/en/get-started/quickstart/hello-world). You can choose to use the GitHub via the terminal (recommanded), the [GitHub desktop app](https://desktop.github.com/), or the online GitHub platform. After setting up your repo, make sure to share your repo with [Jae](https://github.com/jae3goals) and [Xiao](github.com/xshang93) by adding us as collobrators.